In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point
from joblib import Parallel, delayed

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             16G     0   16G   0% /dev
tmpfs           3.2G  784K  3.2G   1% /run
/dev/vda1       155G   41G  115G  26% /
tmpfs            16G  4.0K   16G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            16G     0   16G   0% /sys/fs/cgroup
/dev/vda15      105M  3.6M  101M   4% /boot/efi
/dev/vdb1       2.0T  282G  1.6T  16% /mnt
/dev/vdc1       2.0T  913G  955G  49% /mnt2
tmpfs           3.2G     0  3.2G   0% /run/user/1003


In [3]:
!ls /mnt/raw_data/ | tail

output_20191123.csv
output_20191124.csv
output_20191125.csv
output_20191126.csv
output_20191127.csv
output_20191128.csv
output_20191129.csv
output_20191130.csv
time_binned
total.csv


In [ ]:
np.argsort( )

### Locate directory

In [7]:
source = '/mnt/raw_data/'

In [8]:
files_events = sorted( [ i for i in os.listdir( source ) if 'MT_MTA' in i  ] )
files_events = np.array( [ i for i in files_events if 'EVENTS' in i ] )
files_events[:5], files_events[-5:]

(array(['MT_MTA_NETWORK_EVENTS_20190101.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20190101_0.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20190101_10.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20190101_100.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20190101_1000.csv.gz'], dtype='<U42'),
 array(['MT_MTA_NETWORK_EVENTS_20191126.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20191127.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20191128.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20191129.csv.gz',
        'MT_MTA_NETWORK_EVENTS_20191130.csv.gz'], dtype='<U42'))

### Load events csv

In [9]:
events_df = pd.read_csv( source+files_events[0], delimiter=';', compression='gzip' )

In [10]:
events_df.shape

(102784984, 7)

In [11]:
events_df.head(10)

,event_datetime,equipment_identifier,network_identifier,event_type,event_direction,country_code,device_tac
0,23,5C000000,1598647,S,O,231,1425300.0
1,23,5C000000,1598647,S,I,231,1425300.0
2,1030,5C000000,1598647,S,I,231,1425300.0
3,1955,5C000000,1598647,V,O,231,1425300.0
4,1955,5C000000,1598647,V,O,231,1425300.0
5,1957,5C000000,30547362,V,O,231,1425300.0
6,1957,5C000000,30547362,S,I,231,1425300.0
7,2003,5C000000,1598647,S,I,231,1425300.0
8,2003,5C000000,1598647,V,O,231,1425300.0
9,1255,5C000001,115708964,D,O,216,35975108.0


In [12]:
np.unique( events_df.equipment_identifier.values )

KeyboardInterrupt: 

In [ ]:
np.unique( events_df.event_type.values )

In [ ]:
np.unique( events_df.event_direction.values )

In [ ]:
np.unique( events_df.country_code.values )

#### Time bins

In [13]:
event_time_uq = np.unique( events_df.event_datetime.values )
event_time_uq

array([   0,    1,    2, ..., 2357, 2358, 2359])

In [60]:
%%time
np.where( event_time_uq==events_df.event_datetime[10]  )

CPU times: user 146 µs, sys: 0 ns, total: 146 µs
Wall time: 150 µs


(array([775]),)

In [79]:
list(map( (lambda x, y: np.argwhere( x==y ) ), event_time_uq, events_df.event_datetime[300] ))

TypeError: 'numpy.int64' object is not iterable

In [81]:
%%time
(lambda x, y: np.argwhere( x==y ) )( event_time_uq, events_df.event_datetime[300] )

CPU times: user 393 µs, sys: 0 ns, total: 393 µs
Wall time: 367 µs


array([[1410]])

In [86]:
events_df.event_datetime[300], int(1410/5)

(2330, 282)

In [24]:
event_time_uq[300]

500

In [25]:
time_in_minutes( event_time_uq[300] )

300

In [18]:
def time_in_minutes( time ):
    num_str = str(time)
    num_digits = len(num_str)
    #print(num_str, num_digits)
    if num_digits < 3:
        #print( '2', time )
        return time
    elif num_digits == 3:
        #print( '3', int(num_str[0])*60 + int(num_str[1:]) )
        return int(num_str[0])*60 + int(num_str[1:])
    elif num_digits == 4:
        #print( '4', int(num_str[:2])*60 + int(num_str[2:]) )
        return int(num_str[:2])*60 + int(num_str[2:])
    else:
        print("error!", num_str)
        
def convert_time_to_minutes( events_df, start_idx, end_idx ):
    event_abstime = np.zeros( end_idx-start_idx, dtype=np.int64 )
    event_datetime = events_df.event_datetime.values
    for i in range( start_idx, end_idx ):
        event_abstime[i] =  time_in_minutes( event_datetime[i] )
    return event_abstime

In [19]:
event_abstime = np.zeros( events_df.event_datetime.shape[0], dtype=np.int64 )
event_datetime = events_df.event_datetime.values
for i in tqdm( range( events_df.event_datetime.shape[0] ) ):
    event_abstime[i] =  time_in_minutes( event_datetime[i] ) 
event_abstime.shape

100%|██████████| 102784984/102784984 [03:03<00:00, 559743.06it/s]


(102784984,)

In [14]:
events_df["event_abstime"] = event_abstime

In [11]:
events_df.head(10)

,event_datetime,equipment_identifier,network_identifier,event_type,event_direction,country_code,device_tac,event_abstime
0,23,5C000000,1598647,S,O,231,1425300.0,23
1,23,5C000000,1598647,S,I,231,1425300.0,23
2,1030,5C000000,1598647,S,I,231,1425300.0,630
3,1955,5C000000,1598647,V,O,231,1425300.0,1195
4,1955,5C000000,1598647,V,O,231,1425300.0,1195
5,1957,5C000000,30547362,V,O,231,1425300.0,1197
6,1957,5C000000,30547362,S,I,231,1425300.0,1197
7,2003,5C000000,1598647,S,I,231,1425300.0,1203
8,2003,5C000000,1598647,V,O,231,1425300.0,1203
9,1255,5C000001,115708964,D,O,216,35975108.0,775


In [15]:
abstimes = np.linspace( 0, 1440, int(1440/5)+1 ).astype(int)
abstimes[:5]

array([ 0,  5, 10, 15, 20])

In [21]:
for i in tqdm( range( 0, abstimes.shape[0]-1 ) ):
    time_bin_idx = np.logical_and( abstimes[i] <= events_df.event_abstime, abstimes[i+1] > events_df.event_abstime )
    print( files_events[0].split('.csv')[0]+'_'+str(abstimes[i])+'.csv'+files_events[0].split('.csv')[1] )

  0%|          | 1/288 [00:00<03:57,  1.21it/s]

MT_MTA_NETWORK_EVENTS_20190101_0.csv.gz


  1%|          | 2/288 [00:01<03:13,  1.48it/s]

MT_MTA_NETWORK_EVENTS_20190101_5.csv.gz


  1%|          | 3/288 [00:01<02:39,  1.79it/s]

...

MT_MTA_NETWORK_EVENTS_20190101_1425.csv.gz


100%|█████████▉| 287/288 [01:36<00:00,  2.78it/s]

MT_MTA_NETWORK_EVENTS_20190101_1430.csv.gz


100%|██████████| 288/288 [01:36<00:00,  2.98it/s]

MT_MTA_NETWORK_EVENTS_20190101_1435.csv.gz


In [24]:
events_df[ time_bin_idx ].to_csv(  )

,event_datetime,equipment_identifier,network_identifier,event_type,event_direction,country_code,device_tac,event_abstime
176,2358,5C000012,76926734,D,O,216,35914207.0,1438
2806,2357,5C0000C9,27188118,D,O,216,35289006.0,1437
2974,2356,5C0000D5,238621595,D,O,216,35753708.0,1436
4578,2356,5C000102,1586126,D,O,216,35682300.0,1436
4579,2356,5C000102,1586126,D,O,216,35682300.0,1436
...,...,...,...,...,...,...,...,...
102770570,2357,5C7FFC12,143229699,D,O,216,86689602.0,1437
102776916,2355,5C7FFDC6,1920933,D,O,216,35298709.0,1435
102780277,2358,5C7FFEB2,8045615,D,O,216,35284802.0,1438
102780278,2358,5C7FFEB2,8045615,D,O,216,35284802.0,1438


In [228]:
events_df.network_identifier.values

array([1598647, 1598647, 1598647, ..., 1619669, 1619669, 1619669])

In [230]:
poligons_df.network_identifier

0           10071347
1           10071347
2           10071347
3           10071347
4           10071347
              ...   
25831627    99591498
25831628    99591498
25831629    99591498
25831630    99591498
25831631    99591498
Name: network_identifier, Length: 25831632, dtype: int64

### Load polygons csv

In [12]:
polygons_df = pd.read_csv( source+'MT_MTA_NETWORK_POLIGONS_20190101.csv', delimiter=';' )

In [13]:
polygons_df.shape

(25831632, 6)

In [14]:
polygons_df.head()

,network_identifier,eovx,eovy,dx,dy,network_element_type
0,10071347,216918,496318,127,127,D
1,10071347,216918,496445,127,127,D
2,10071347,217045,496318,127,127,D
3,10071347,217045,496445,127,127,D
4,10071347,217172,496191,127,127,D


In [26]:
net_id_uq = np.unique( polygons_df.network_identifier.values )
net_id_uq.shape, net_id_uq[:20]

((42723,),
 array([ 71892,  72307,  72922,  73325, 140903, 146953, 201108, 201256,
        201465, 206489, 206823, 206923, 228930, 229021, 229146, 262799,
        263008, 263131, 263658, 263851]))

In [27]:
el_typ_uq = np.unique( polygons_df.network_element_type.values )
el_typ_uq.shape, el_typ_uq

((6,), array(['A', 'B', 'C', 'D', 'E', 'U'], dtype=object))

In [38]:
np.unique( polygons_df.dx.values ), np.unique( polygons_df.dy.values )

(array([127]), array([127]))

#### There are 42723 towers

In [253]:
points_np = np.vstack( ( poligons_df.eovx.values, poligons_df.eovy.values ) ).T

points = []
for i in tqdm( range( points_np.shape[0] ) ):
    points.append( Point(points_np[i][0], points_np[i][1]) ) 

100%|██████████| 25831632/25831632 [03:47<00:00, 113574.50it/s]


In [ ]:
series = gpd.GeoSeries(points)
geo = gpd.GeoDataFrame({'geo': series}, geometry='geo')
geo.crs = {'init': 'epsg:23700'}
geo = geo.to_crs(epsg=4326)
geo.head()

/home/biricza/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [28]:
polygons_df[ np.in1d( polygons_df.network_identifier.values, net_id_uq[0] ) ]

,network_identifier,eovx,eovy,dx,dy,network_element_type
22857738,71892,245366,552579,127,127,B
22857739,71892,245493,552579,127,127,B
22857740,71892,245493,552706,127,127,B
22857741,71892,245620,552452,127,127,B
22857742,71892,245620,552579,127,127,B
...,...,...,...,...,...,...
22861689,71892,259082,556262,127,127,B
22861690,71892,259082,560072,127,127,B
22861691,71892,259082,560199,127,127,B
22861692,71892,259209,559945,127,127,B


In [90]:
from shapely.geometry import Polygon, box

In [137]:
tower_poly_eov = polygons_df[ np.in1d( polygons_df.network_identifier.values, net_id_uq[0] ) ].values[:,1:3]
tower_poly_eov_coords = np.concatenate( ( 
                  np.vstack( (tower_poly_eov[:,0]-63, tower_poly_eov[:,1]-63) ).T, 
                  np.vstack( (tower_poly_eov[:,0]-63, tower_poly_eov[:,1]+64) ).T,
                  np.vstack( (tower_poly_eov[:,0]+64, tower_poly_eov[:,1]-63) ).T,
                  np.vstack( (tower_poly_eov[:,0]+64, tower_poly_eov[:,1]+64) ).T ), axis=1 )
tower_poly_eov_coords.shape #.reshape(4, -1, 2)

(3956, 8)